# Modeling

I am using Color Refinement to extract graph level features and a MLP to train a 

[Here is a demo of Color Refinement](https://github.com/parkerburchett/TDC-DeepLearning/blob/main/utils/ColorRefinement_tutorial.ipynb)


In [1]:
!pip install pysmiles
!pip install pyTDC
!git clone https://github.com/parkerburchett/TDC-DeepLearning

  Using cached pbr-5.6.0-py2.py3-none-any.whl (111 kB)
  Created wheel for pysmiles: filename=pysmiles-1.0.1-py2.py3-none-any.whl size=22028 sha256=346f0e1c39d68b729d65bedb3f93b8cb948c83d0ded9c310aada0414454c877f
  Stored in directory: /root/.cache/pip/wheels/e4/f0/ca/dae2e932684a6e26824d29cf5b6dadea7320e6fed036942972
Successfully built pysmiles
     |████████████████████████████████| 85 kB 3.4 MB/s 
  Created wheel for pyTDC: filename=PyTDC-0.3.1-py3-none-any.whl size=116110 sha256=d3fb24e966248e4c352fdd1af8c2516757e4c9a2aefdf934bad4d9d1305d42cc
  Stored in directory: /root/.cache/pip/wheels/a9/54/07/50251965a66a68eb6c0e2b3022588cc992cc4c2a2e69d8c7ec
Successfully built pyTDC
Cloning into 'TDC-DeepLearning'...
remote: Enumerating objects: 210, done.
remote: Counting objects: 100% (210/210), done.
remote: Compressing objects: 100% (170/170), done.
remote: Total 210 (delta 63), reused 182 (delta 35), pack-reused 0
Receiving objects: 100% (210/210), 48.42 MiB | 24.29 MiB/s, done.
Resolvin

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import networkx as nx
import tensorflow as tf
from sklearn.metrics import precision_recall_curve, auc
from tensorflow.keras.optimizers import SGD
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import pickle
from pysmiles import read_smiles
from tdc.single_pred import ADME
import os
os.chdir('/content/TDC-DeepLearning/')
from utils import ColorRefinement, utils

Mounted at /content/drive


In [3]:
color_graphs = pickle.load(open('/content/drive/MyDrive/SpringBoard/Therapeutic Data Commons Projects/data/color_graphs.pkl', 'rb'))

# This is the 0-10 hop dfs of each of the molecules. Since it takes 45 minutes to run I ran it once and am just loading it each time

In [7]:
color_graphs[0]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4960,4961,4962,4963,4964,4965,4966,4967,4968,4969,4970,4971,4972,4973,4974,4975,4976,4977,4978,4979,4980,4981,4982,4983,4984,4985,4986,4987,4988,4989,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
iteration_number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
color_0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
color_1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
color_2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
color_3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
color_4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
color_5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
color_6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
color_7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
color_8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


All of the feautre vectors are very spare, typically 95% + zeros.

In [5]:
def get_n_dem_embedding(color_graphs, n):
  embeddings = []
  for g in color_graphs:
    embeddings.append(g.values[n,:])
  return pd.DataFrame(np.array(embeddings))


In [6]:
feature_dfs = [get_n_dem_embedding(color_graphs, n) for n in range(0,10)]

In [9]:
feature_dfs[0]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4960,4961,4962,4963,4964,4965,4966,4967,4968,4969,4970,4971,4972,4973,4974,4975,4976,4977,4978,4979,4980,4981,4982,4983,4984,4985,4986,4987,4988,4989,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12660,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12661,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12662,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12663,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Each row is a molecule, and each colum is a count of the final colors of each of it's nodes.

In [12]:
feature_dfs[0].sum(axis=0).sort_values().tail(10)

3327     8731.0
3398     9516.0
2357    11544.0
4882    12734.0
3448    18956.0
4073    25399.0
3321    28378.0
3723    41797.0
884     50259.0
2659    78256.0
dtype: float64

The O hop assigned each unique kind of node, (unique element, h_count, aromatic, and charge) -> a unique color. There are no false negatives, there is a 1/5000 chance of a false positive.
 
Otherwise, if you see the number 2659 assigned to different nodes in two molecules you can be very confident that those nodes are identical


In [13]:
feature_dfs[3].sum(axis=0).sort_values().tail(10)

4955     945.0
4258    1075.0
42      1080.0
4769    1096.0
1058    1386.0
285     1390.0
81      1406.0
2775    1441.0
450     2186.0
982     3922.0
dtype: float64

The 3 refers to the 3 hop neighborhoods, this means that when you see two nodes having the color 982 there is a 5000-(3+1)/5000 chance that the local 3 hop neighborhood around those nodes is identical.
 
I am using this method to convert arbitrarily sized local neighborhoods into features.
 
The 3 hop neighborhood converts each 3 hop neighborhood into a color and then a graph is encoded as a count of each of the [0, 4999] possible colors.
 


In [ ]:
# for index, df in enumerate(feature_dfs):
#   df.to_csv(f'/content/drive/MyDrive/SpringBoard/Therapeutic Data Commons Projects/data/feature_hop_{index}.csv')
#   print('saved a df')

In [8]:
df = pd.read_csv('/content/TDC-DeepLearning/data/cleaned_raw_data.csv')
df

,SMILE,Target
0,CCCC(=O)Nc1ccc(N2CCN(CC)CC2)c(Cl)c1.Cl,0
1,CC(=O)N(c1ccc2oc(=O)sc2c1)S(=O)(=O)c1cccs1,1
2,Clc1ccccc1-c1nc(-c2ccccc2)n[nH]1,1
3,COc1ccccc1C(c1nnnn1C(C)(C)C)N1CCN(Cc2ccncc2)CC1,1
4,COc1ccccc1CNC(=O)Cn1nnc(-c2ccncc2)n1,1
...,...,...
12660,CCN1C(=O)[C@@H]2[C@@H](CC[C@@H]3C(=O)C=C[C@@H]...,0
12661,O=C1[C@H]2CC=C3[C@@H]([C@H](O)[C@H]4O[C@@H]4C3...,0
12662,CCN1C(=O)[C@H]2CC=C3[C@@H]([C@H](O)[C@H]4O[C@@...,1
12663,C[C@H](c1ccccc1)N1C(=O)[C@@H]2[C@@H](CC[C@@H]3...,0


In [14]:
features = pd.concat(feature_dfs[3:], axis=1).astype(np.int8)
# using the colors of hop 3 to 10
print(features.dtypes[0])
print(features.shape)


0    int8
0    int8
0    int8
0    int8
0    int8
0    int8
0    int8
dtype: object
(12665, 35000)


Each molecule is converted into a vector of the 3 hop neighborhoods appended to the 4 hop neighborhoods ... 10 hop neighborhood. That vector is then passed into the MLP.

In [15]:
def compute_auprc(model, X_test, y_test):
    # https://stats.stackexchange.com/questions/157012/area-under-precision-recall-curve-auc-of-pr-curve-and-average-precision-ap
    precision, recall, thresholds = precision_recall_curve(y_test, model.predict(X_test))
    area = auc(recall, precision)
    return area

def get_optimizer():
  # https://towardsdatascience.com/simple-guide-to-hyperparameter-tuning-in-neural-networks-3fe03dad8594

  # I have not tuned these parameters
	learning_rate = 0.1
	decay_rate = learning_rate / 300
	momentum = 0.8
	sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
	return sgd

  
def create_baseline():
  # https://towardsdatascience.com/simple-guide-to-hyperparameter-tuning-in-neural-networks-3fe03dad8594
  # create model
  model = Sequential()
  
  # I have not tuned these. I just picked what seemed reasonable. 
  model.add(Dense(1500, input_dim=features.shape[1], activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  # Compile model

  model.compile(loss='binary_crossentropy', optimizer=get_optimizer(), metrics=['accuracy'])
  return model

In [16]:
X, y = features.values, df['Target'].values

for i in range(5):
  
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=i)
  for s in range(5):
    model = create_baseline()
    model.fit(X_train, y_train, validation_data=(X_test, y_test))
    score = compute_auprc(model, X_test, y_test)
    print(f'AUPRC : {score}')


317/317 [==============================] - 6s 8ms/step - loss: 0.5702 - accuracy: 0.6955 - val_loss: 0.4917 - val_accuracy: 0.7702
AUPRC : 0.822865526719467
317/317 [==============================] - 3s 8ms/step - loss: 0.5697 - accuracy: 0.6922 - val_loss: 0.5325 - val_accuracy: 0.7529
AUPRC : 0.8037865212272748
317/317 [==============================] - 3s 8ms/step - loss: 0.5779 - accuracy: 0.6817 - val_loss: 0.5097 - val_accuracy: 0.7414
AUPRC : 0.8101666812086236
317/317 [==============================] - 3s 8ms/step - loss: 0.5756 - accuracy: 0.6888 - val_loss: 0.5024 - val_accuracy: 0.7687
AUPRC : 0.8020283695215262
317/317 [==============================] - 3s 8ms/step - loss: 0.5762 - accuracy: 0.6880 - val_loss: 0.5195 - val_accuracy: 0.7363
AUPRC : 0.8067932166648991
317/317 [==============================] - 3s 8ms/step - loss: 0.5655 - accuracy: 0.7012 - val_loss: 0.5294 - val_accuracy: 0.7288
AUPRC : 0.7808676507972744
317/317 [==============================] - 3s 8ms/ste

# Outcomes

AUPRC is usally between ~77% and ~82%.
